In [1]:
# Basic imports
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
import wandb

In [3]:
# Download data and create dataloaders
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

# set up class labels
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
import torch.nn as nn
import torch.nn.functional as F

# Define Model
class Net(nn.Module):
    def __init__(self, channels_1, channels_2, hidden_1, hidden_2):
        super().__init__()
        self.conv1 = nn.Conv2d(3, channels_1, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(channels_1, channels_2, 5)
        self.fc1 = nn.Linear(channels_2 * 5 * 5, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define train config
config = {
    "model_init_kwargs": {
        "channels_1": 6,
        "channels_2": 16,
        "hidden_1": 128,
        "hidden_2": 64
    },
    "optimizer_init_kwargs": {
        "lr": 1e-3,
        "momentum": 0.9
    },
    "criterion": nn.CrossEntropyLoss
}

wandb.init(project="wandb-demo-gp",
           config=config)
net = Net(**config["model_init_kwargs"])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: till-m. Use `wandb login --relogin` to force relogin


In [5]:
import torch.optim as optim

# Define loss criterion
criterion = config["criterion"]()
# Define & configure optimizer
optimizer = optim.SGD(net.parameters(), **config["optimizer_init_kwargs"])

In [6]:
# Plot a few images from the dataset
import matplotlib.pyplot as plt
import numpy as np

# function to show an image
def imshow(img, lbls, category):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    wimg = wandb.Image(np.transpose(npimg, (1, 2, 0)), caption=' '.join(f'{classes[lbl]:5s}' for lbl in lbls))
    wandb.log({category + "/images": wimg})


In [7]:
# Main training loop

for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        wandb.log({"train/batch_loss": loss.item()})
        if i % 2000 == 1999:    # print every 2000 mini-batches
            wandb.log({"train/running_loss": running_loss / 2000})
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

    # Calculate accuracy of model on test set

    # prepare to count predictions for each class
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}

    # again no gradients needed
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predictions = torch.max(outputs, 1)
            # collect the correct predictions for each class
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1


    correct = 0
    total = 0
    # print accuracy for each class
    for classname, correct_count in correct_pred.items():
        correct += correct_count
        total += total_pred[classname]
        accuracy = 100 * float(correct_count) / total_pred[classname]
        wandb.log({f"test/accuracy/{classname}": accuracy})

    wandb.log({"test/accuracy": 100 * correct // total})
    print(f'Total accuracy of the network on the 10000 test images: {100 * correct // total} %')

    # Predictions on text set
    dataiter = iter(testloader)
    images, labels = next(dataiter)
    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    # print images
    imshow(torchvision.utils.make_grid(images), predicted, "test")
print('Finished Training')

[1,  2000] loss: 2.207
[1,  4000] loss: 1.869
[1,  6000] loss: 1.696
[1,  8000] loss: 1.606
[1, 10000] loss: 1.511
[1, 12000] loss: 1.461
Total accuracy of the network on the 10000 test images: 47 %
[2,  2000] loss: 1.401
[2,  4000] loss: 1.377
[2,  6000] loss: 1.353
[2,  8000] loss: 1.320
[2, 10000] loss: 1.300
[2, 12000] loss: 1.285
Total accuracy of the network on the 10000 test images: 55 %
[3,  2000] loss: 1.210
[3,  4000] loss: 1.208
[3,  6000] loss: 1.200
[3,  8000] loss: 1.194
[3, 10000] loss: 1.175
[3, 12000] loss: 1.135
Total accuracy of the network on the 10000 test images: 58 %
[4,  2000] loss: 1.088
[4,  4000] loss: 1.085
[4,  6000] loss: 1.097
[4,  8000] loss: 1.096
[4, 10000] loss: 1.099
[4, 12000] loss: 1.075
Total accuracy of the network on the 10000 test images: 59 %
[5,  2000] loss: 0.989
[5,  4000] loss: 1.019
[5,  6000] loss: 0.997
[5,  8000] loss: 1.031
[5, 10000] loss: 1.035
[5, 12000] loss: 1.058
Total accuracy of the network on the 10000 test images: 61 %
[6,  

In [8]:
# Save the trained model
path_to_model = 'models/cifar10_model.pt'
torch.save(net, path_to_model)

In [9]:
artifact = wandb.Artifact('net', type='model')
artifact.add_file(path_to_model)
wandb.log_artifact(artifact)

In [10]:
wandb.finish()